In [121]:
import pandas as pd
import numpy as np

In [122]:
impressions_df = pd.read_csv("../datasets/impressions/2023/impressions_01_05_brands.csv")
impressions_df

,id,hostname,beacon_ids,first_event_at_iso,ad_width_chosen,ad_height_chosen,placement_ids,placement_ids_chosen,total_fixation_duration,channel,beacon_type_chosen,ad_technical_format,is_fixated,gaze_valid,is_iab_inview,exist_viewable_1_s_threshold_50,exist_viewable_2_s_threshold_50,part_month,part_year,chosen_brand
0,a36a0f9a-1a67-4c83-a313-5d2899e1d262,classic.prisjakt.nu,"[""6ac7b3b3-19be-40eb-b46e-48397bcdd651""]",2023-02-19T05:08:57.685Z,285,230,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,99,display,ad/doubleclick.net-iframe-v1,display,True,True,True,True,True,2,2023,netonnet
1,d8b122f0-a707-40cd-ba94-79c775cba07e,cointiply.com,"[""2a579515-79ba-44e6-a9a8-66069bcea6cd""]",2023-02-19T05:45:15.915Z,728,90,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/criteo.com-iframe-v1,display,False,False,False,False,False,2,2023,samsung
2,6da9d257-1d33-4c38-b043-e009f25ba3b9,wrestlezone.com,"[""34802d95-f215-4591-8e2f-32e9854e66f5"",""b2b35...",2023-02-19T05:26:44.431Z,250,480,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/doubleclick-img-link-google-v1,display,False,False,True,True,True,2,2023,sodastream
3,21ae539c-b0f2-4fbf-8c43-4561e70c8cc9,placera.se,"[""25bfdb27-cb13-4104-95be-ed4d4f8215a5""]",2023-01-31T16:09:35.295Z,980,240,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/doubleclick.net-iframe-v1,display,False,False,True,True,True,1,2023,telia
4,101caa66-f19c-40dd-9d83-28719ce0dcac,facebook.com,"[""72ffbcd9-3d94-47dc-9ba4-cc830295fce0""]",2023-01-31T16:15:26.926Z,285,230,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,social,ad/facebook.com-sidebar-ads-v1,display,False,True,False,False,False,1,2023,bmw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305299,692dffbd-39b3-4401-8348-9c108196e311,youtube.com,"[""60d26c03-9a30-4ec9-bb6f-df0143f2e9f4""]",2023-04-12T16:59:26.886Z,180,150,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/youtube.com-watch-sidebar-featured-v1,display,False,False,False,False,False,4,2023,skoda
1305300,089dedfa-2e9e-479a-b684-267d6207ebdd,youtube.com,"[""31d74e22-6724-4fbd-88b1-17d21b592abe""]",2023-04-12T16:34:37.898Z,180,150,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/youtube.com-watch-sidebar-featured-v1,display,False,False,False,False,False,4,2023,tui
1305301,9a4e4567-6ee1-41b2-a0b7-d3c7ee7a1082,youtube.com,"[""6eb590de-f5e8-4324-b4af-d8929813486f"",""e0427...",2023-04-12T16:27:08.314Z,468,120,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/youtube.com-watch-companion-v1,display,False,False,False,False,False,4,2023,cmore
1305302,aa59325a-8e58-4e05-8b11-6d264dd1f20e,svd.se,"[""1d46f1a8-801a-4c8c-9d58-69e23fd6ae97"",""2ff3c...",2023-04-12T16:58:13.874Z,1180,240,"{""tag_id"":[""19499726""],""bam_ad_slot"":["""","""",""""]}",NaN,1234,display,ad/newssites-banners-v1,display,True,True,True,True,True,4,2023,amex


In [123]:
import json

def extract_pid(placement_id):
    placement_id = json.loads(placement_id)
    bam_ad_slots = placement_id["bam_ad_slot"]
    # print(bam_ad_slots)
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    ret = ''
    try:
        ret = placement_id['tag_id'][0]
    except:
        # print('.')
        ret = ','.join([bas for bas in final_bam_ad_slots])
    else:
        ret += ','.join([bas for bas in final_bam_ad_slots])
    return ret


def pid_type(pid):
    try:
        int(pid)
        return 'int'
    except:
        return 'str'

impressions_df['pid'] = impressions_df['placement_ids'].apply(extract_pid)
impressions_df['pid_type'] = impressions_df['pid'].apply(pid_type)
# filter only impressions with int placement IDs
impressions_df = impressions_df[impressions_df['pid_type'] == 'int']
# convert placement ids that can be converted to the int to be int, not string
# type(impressions_df.iloc[0]['pid'])     # check this, right now they're strings
impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))

/tmp/ipykernel_24348/1786883881.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))


In [124]:
# run through predictions
import os
import json


def _get_pred_map():
    pred_map = dict()
    for i in range(1, 6):
        if i < 10:
            i = f'0{i}'
        for day in os.listdir(f'../predictions/2023/{i}'):
            for h in os.listdir(f'../predictions/2023/{i}/{day}'):
                if not h.endswith('.ndjson'):
                    continue
                with open(f'../predictions/2023/{i}/{day}/{h}')as f:
                    for line in f.readlines():
                        json_line = json.loads(line)
                        id_ = json_line['id']
                        pred_map[id_] = json_line['prediction']
    return pred_map
                            
pred_map = _get_pred_map()

In [125]:
def _get_final_fixation(gaze_valid, is_fixated, id_, pred_map):
    if gaze_valid:
        return is_fixated
    if id_ in pred_map:
        return pred_map[id_]
    return False

impressions_df['final_fixation'] = impressions_df.apply(lambda row: _get_final_fixation(row['gaze_valid'], row['is_fixated'], row['id'], pred_map), axis=1)
impressions_df

/tmp/ipykernel_24348/3603812544.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impressions_df['final_fixation'] = impressions_df.apply(lambda row: _get_final_fixation(row['gaze_valid'], row['is_fixated'], row['id'], pred_map), axis=1)


,id,hostname,beacon_ids,first_event_at_iso,ad_width_chosen,ad_height_chosen,placement_ids,placement_ids_chosen,total_fixation_duration,channel,...,gaze_valid,is_iab_inview,exist_viewable_1_s_threshold_50,exist_viewable_2_s_threshold_50,part_month,part_year,chosen_brand,pid,pid_type,final_fixation
6,4e33f956-e2e5-456e-b4ce-8dba4b98cd7d,aftonbladet.se,"[""230454e4-703a-4de1-bac1-ae040dd2cdae""]",2023-01-31T16:24:32.657Z,180,150,"{""tag_id"":[""19499407""],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,...,True,False,False,False,1,2023,lendo,19499407,int,False
7,4e33f956-e2e5-456e-b4ce-8dba4b98cd7d,aftonbladet.se,"[""230454e4-703a-4de1-bac1-ae040dd2cdae""]",2023-01-31T16:24:32.657Z,180,150,"{""tag_id"":[""19499407""],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,...,True,False,False,False,1,2023,lendototal,19499407,int,False
30,a1e470d2-0a51-427b-9608-439536a3b86c,blocket.se,"[""263d8033-5883-478c-a400-e1d02b6591d8""]",2023-04-01T17:58:32.786Z,250,480,"{""tag_id"":[""19499525""],""bam_ad_slot"":["""","""",""""]}",NaN,11,display,...,True,True,True,True,4,2023,komplett,19499525,int,False
31,a1e470d2-0a51-427b-9608-439536a3b86c,blocket.se,"[""263d8033-5883-478c-a400-e1d02b6591d8""]",2023-04-01T17:58:32.786Z,250,480,"{""tag_id"":[""19499525""],""bam_ad_slot"":["""","""",""""]}",NaN,11,display,...,True,True,True,True,4,2023,amazon,19499525,int,False
38,ddc34dbd-18f1-4588-bf70-d0d08af77446,aftonbladet.se,"[""1dcb798f-6573-4ed0-bf31-26009fdef028"",""33db1...",2023-04-04T04:19:14.754Z,250,480,"{""tag_id"":[""19499420""],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,...,False,False,False,False,4,2023,elon,19499420,int,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305294,0f30e711-35ac-497d-8b73-24e0ba817029,aftonbladet.se,"[""c863a1c7-2b09-4109-9201-9991fe63275c""]",2023-04-12T16:49:04.266Z,300,250,"{""tag_id"":[""19499424""],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,...,True,False,False,False,4,2023,tui,19499424,int,False
1305295,460df5e3-adc8-4a17-a208-99c4a0d0c939,svd.se,"[""22d0f0a2-ef35-4096-b333-f85d46dd9e2b"",""2db66...",2023-04-12T16:51:47.202Z,1180,240,"{""tag_id"":[""19499705""],""bam_ad_slot"":["""","""",""""]}",NaN,122,display,...,True,False,False,False,4,2023,samsung,19499705,int,True
1305296,460df5e3-adc8-4a17-a208-99c4a0d0c939,svd.se,"[""22d0f0a2-ef35-4096-b333-f85d46dd9e2b"",""2db66...",2023-04-12T16:51:47.202Z,1180,240,"{""tag_id"":[""19499705""],""bam_ad_slot"":["""","""",""""]}",NaN,122,display,...,True,False,False,False,4,2023,telia,19499705,int,True
1305298,8b84f5db-6c3a-4638-acc5-9b77be7e5274,svd.se,"[""254dc172-6261-42e3-80eb-51c44bbcd4e7"",""26299...",2023-04-12T16:56:07.429Z,980,240,"{""tag_id"":[""19499712""],""bam_ad_slot"":["""","""",""""]}",NaN,0,out-stream,...,True,True,True,True,4,2023,tre,19499712,int,False


In [126]:
impressions_df["true_is_iab_inview"] = impressions_df.apply(
    lambda x: 
    True if (
        (x["ad_technical_format"] == "out-stream") & (x["exist_viewable_1_s_threshold_50"] == True) & (x["exist_viewable_2_s_threshold_50"] == False)
    ) | (x["is_iab_inview"] == True) 
    else False, 
    axis=1
)

/tmp/ipykernel_24348/293613858.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impressions_df["true_is_iab_inview"] = impressions_df.apply(


In [127]:
client = "Jula"
tier = "3"
client_impressions = pd.read_excel(f"../datasets/{client}.xlsx", sheet_name="desktop")
client_impressions["Placement Id"] = client_impressions["Placement Id"].astype("Int64")
client_impressions = client_impressions.dropna(subset=["Placement Id"])

In [128]:
if tier in ("1", "2"):
    impressions_df = impressions_df[impressions_df["chosen_brand"] == client.lower()]
if tier == "1":
    impressions_df = impressions_df[impressions_df["part_month"] == 5]

In [129]:
grouped_df = impressions_df.groupby(["hostname", "pid"]).apply(
    lambda x: pd.Series(
        {
            "impressions": x["id"].count(),
            "fixations": x.loc[x["final_fixation"], "id"].count(),
            "inview": x.loc[x["true_is_iab_inview"], "id"].count()
        }
    )
)
grouped_df = grouped_df.reset_index()

In [130]:
merged_df = grouped_df.merge(client_impressions, how="inner", left_on="pid", right_on="Placement Id")

In [131]:
final_df = merged_df[["hostname", "Placement Id", "impressions", "fixations", "inview", "Imps", "Viewability Rate %"]]

In [132]:
final_group = final_df.groupby(["hostname"]).apply(
     lambda x: pd.Series(
         {
             "tobii_imps": x["impressions"].sum(),
             "client_imps": x["Imps"].sum(),
             "tobii_fixations": x["fixations"].sum(),
             "tobii_inview": x["inview"].sum(),
             "client_viewability_rate": np.average(x["Viewability Rate %"], weights=x["Imps"])
         }
     )
)
final_group = final_group.reset_index()
final_group["tobii_imps"] = final_group["tobii_imps"].astype("Int64")
final_group["client_imps"] = final_group["client_imps"].astype("Int64")
final_group["tobii_fixations"] = final_group["tobii_fixations"].astype("Int64")
final_group["tobii_inview"] = final_group["tobii_inview"].astype("Int64")
final_group["tobii_inview/impression_ratio"] = round(final_group["tobii_inview"] / final_group["tobii_imps"] * 100, 2)
final_group["tobii_fixation/impression_ratio"] = round(final_group["tobii_fixations"] / final_group["tobii_imps"] * 100, 2)

In [133]:
final_group["client_inview"] = np.round(final_group["client_imps"] * final_group["client_viewability_rate"]).astype("Int64")
final_group["client_fixations"] = np.round((final_group["tobii_fixations"] / final_group["tobii_imps"]) * final_group["client_imps"]).astype("Int64")
final_group["client_fix/inview_ratio"] = round(final_group["client_fixations"] / final_group["client_inview"] * 100, 2)

In [134]:
def get_sample_size(num_impressions):
    if num_impressions <= 99:
        return "low"
    elif num_impressions >= 100 and num_impressions <= 199:
        return "medium"
    else:
        return "high"

final_group["sample_size"] = final_group["tobii_imps"].apply(lambda x: get_sample_size(x))

In [135]:
result_df = final_group[["hostname", "tobii_imps", "client_imps", "tobii_fixations", "client_fixations", "tobii_inview", "client_inview", "client_fix/inview_ratio", "tobii_inview/impression_ratio", "client_viewability_rate", "tobii_fixation/impression_ratio", "sample_size"]]
result_df
result_df.to_excel(f"../temp/{client.lower()}_tier_{tier}_example_report_FINAL.xlsx", index=False)